The aim of this notebook is to illustrate the usage of *wnpoly*.  Begin by installing the package.

In [1]:
import sys
!{sys.executable} -m pip install --quiet numpy
!{sys.executable} -m pip install --quiet scipy
!{sys.executable} -m pip install --quiet wnpoly

Next import the packages.

In [2]:
import numpy as np
import wnpoly as wp

*wnpoly* currently has two modules.  The first handles [Symmetric polynomials](#symmetric), which are polynomials that remain unchanged under exchange of any variables.  This [Wikipedia page](https://en.wikipedia.org/wiki/Symmetric_polynomial) provides an introduction.  The second handles [Bell polynomials](#bell), which appear particularly in problems involving [set partitions](https://en.wikipedia.org/wiki/Bell_polynomials).

<a id='symmetric'></a>
# Symmetric polynomials

This section illustrates *wnpoly's* symmetric polynomial classes, which include [complete homogeneous symmetric](https://en.wikipedia.org/wiki/Complete_homogeneous_symmetric_polynomial), [elementary homogeneous symmetric](https://en.wikipedia.org/wiki/Elementary_symmetric_polynomial), and [power sum symmetric](https://en.wikipedia.org/wiki/Power_sum_symmetric_polynomial) polynomials.

Begin by creating instances of each class.

In [3]:
chsp = wp.complete()
ehsp = wp.elementary()
pssp = wp.power_sum()

Now create a set of variables *x*.  Here we choose a set of ten randomly chosen integers between 1 and 10.  By definition, *x[0]* should equal 0, so we insert that at the beginning of the array; thus, the array has 11 elements with index ranging from 0 to 10.  In fact, the *wnpoly* routines do not use *x[0]* in computations but rather use it as a placeholder to allow the index of the array to correspond to the expected *x* or polynomial.

In [4]:
n = 10
x = np.random.default_rng().integers(1, high=10, size=n)
x = np.insert(x, 0, 0)

Now compute the corresponding set of polynomials of the various types up to order *n*.

In [5]:
h = chsp.compute(x, n)
e = ehsp.compute(x, n)
p = pssp.compute(x, n)

Print out the values and the *x*'s.

In [6]:
for n in range(len(x)):
    print("n = {:d}, x[n] = {:d}, h[n] = {:d}, e[n] = {:d}, p[n] = {:d}".format(n, x[n], h[n], e[n], p[n]))

n = 0, x[n] = 0, h[n] = 1, e[n] = 1, p[n] = 1
n = 1, x[n] = 9, h[n] = 51, e[n] = 51, p[n] = 51
n = 2, x[n] = 1, h[n] = 1478, e[n] = 1123, p[n] = 355
n = 3, x[n] = 3, h[n] = 32084, e[n] = 13979, p[n] = 2769
n = 4, x[n] = 9, h[n] = 581214, e[n] = 108205, p[n] = 22663
n = 5, x[n] = 5, h[n] = 9294058, e[n] = 539969, p[n] = 190161
n = 6, x[n] = 5, h[n] = 135663644, e[n] = 1743657, p[n] = 1619215
n = 7, x[n] = 8, h[n] = 1847470632, e[n] = 3562961, p[n] = 13918809
n = 8, x[n] = 2, h[n] = 23826689479, e[n] = 4367814, p[n] = 120435943
n = 9, x[n] = 8, h[n] = 294170407741, e[n] = 2881440, p[n] = 1047202881
n = 10, x[n] = 1, h[n] = 3504695807658, e[n] = 777600, p[n] = 9140643775


We can now check some of [Newton's identities](https://en.wikipedia.org/wiki/Newton%27s_identities).  The first we will check relates the complete and elementary polynomials such that $d_n = \sum_{k = 0}^n (-1)^k h_{n-k}(x_1, ..., x_n) e_k(x_1, ..., x_n) = 0$.

In [7]:
for n in range(1, len(x)):
    d = 0
    for k in range(n+1):
        d += np.power(-1, k) * h[n-k] * e[k]
    print("n = {:d}, d[n] = {:d}".format(n, d))

n = 1, d[n] = 0
n = 2, d[n] = 0
n = 3, d[n] = 0
n = 4, d[n] = 0
n = 5, d[n] = 0
n = 6, d[n] = 0
n = 7, d[n] = 0
n = 8, d[n] = 0
n = 9, d[n] = 0
n = 10, d[n] = 0


Now we check an identity relating the complete homogeneous symmetric and power sum symmetric polynomials, namely, that $nh_n = \sum_{k=1}^n h_{n-k} p_k$.

In [8]:
for n in range(1, len(x)):
    sum = 0
    for k in range(1, n+1):
        sum += h[n-k] * p[k]
    print(
        "n = {:d}, n*h[n] = {:d}, sum = {:d}, difference = {:d}".format(
            n, n*h[n], sum, n*h[n] - sum)
    )

n = 1, n*h[n] = 51, sum = 51, difference = 0
n = 2, n*h[n] = 2956, sum = 2956, difference = 0
n = 3, n*h[n] = 96252, sum = 96252, difference = 0
n = 4, n*h[n] = 2324856, sum = 2324856, difference = 0
n = 5, n*h[n] = 46470290, sum = 46470290, difference = 0
n = 6, n*h[n] = 813981864, sum = 813981864, difference = 0
n = 7, n*h[n] = 12932294424, sum = 12932294424, difference = 0
n = 8, n*h[n] = 190613515832, sum = 190613515832, difference = 0
n = 9, n*h[n] = 2647533669669, sum = 2647533669669, difference = 0
n = 10, n*h[n] = 35046958076580, sum = 35046958076580, difference = 0


Finally, we will check the identity that $n e_n = \sum_{k=1}^n (-1)^{k-1} e_{n-k} p_k$.

In [9]:
for n in range(1, len(x)):
    sum = 0
    for k in range(1, n+1):
        sum += np.power(-1, k-1) * e[n-k] * p[k]
    print(
        "n = {:d}, n*e[n] = {:d}, sum = {:d}, difference = {:d}".format(
            n, n*e[n], sum, n*e[n] - sum)
    )

n = 1, n*e[n] = 51, sum = 51, difference = 0
n = 2, n*e[n] = 2246, sum = 2246, difference = 0
n = 3, n*e[n] = 41937, sum = 41937, difference = 0
n = 4, n*e[n] = 432820, sum = 432820, difference = 0
n = 5, n*e[n] = 2699845, sum = 2699845, difference = 0
n = 6, n*e[n] = 10461942, sum = 10461942, difference = 0
n = 7, n*e[n] = 24940727, sum = 24940727, difference = 0
n = 8, n*e[n] = 34942512, sum = 34942512, difference = 0
n = 9, n*e[n] = 25932960, sum = 25932960, difference = 0
n = 10, n*e[n] = 7776000, sum = 7776000, difference = 0


It is also possible to compute the symmetric polynomials normalized by the number of terms in each polynomial.  Here one should not use integers but rather floats or similar type.  Again, the first *x* should be zero, so insert that.

In [10]:
n = 5
x = np.random.default_rng().uniform(0.0, 10.0, size=n)
x = np.insert(x, 0, 0.)

Now compute and print out the normalized polynomials.

In [11]:
h_norm = chsp.compute_normalized(x, n)
e_norm = ehsp.compute_normalized(x, n)
p_norm = pssp.compute_normalized(x, n)

for n in range(len(x)):
    print(
        "n = {:d}, x[n] = {:.4f}, h_norm[n] = {:.4f}, e_norm[n] = {:.4f}, p_norm[n] = {:.4f}".format(
        n, x[n], h_norm[n], e_norm[n], p_norm[n]))

n = 0, x[n] = 0.0000, h_norm[n] = 1.0000, e_norm[n] = 1.0000, p_norm[n] = 1.0000
n = 1, x[n] = 4.9140, h_norm[n] = 3.7950, e_norm[n] = 3.7950, p_norm[n] = 3.7950
n = 2, x[n] = 2.6837, h_norm[n] = 15.0749, e_norm[n] = 13.3932, p_norm[n] = 18.4382
n = 3, x[n] = 4.1541, h_norm[n] = 62.2240, e_norm[n] = 42.8504, p_norm[n] = 98.6947
n = 4, x[n] = 6.5694, h_norm[n] = 265.4662, e_norm[n] = 117.5890, p_norm[n] = 559.1008
n = 5, x[n] = 0.6539, h_norm[n] = 1165.8629, e_norm[n] = 235.3317, p_norm[n] = 3295.5431


<a id='bell'></a>
# Bell polynomials

This section illustrates *wnpoly's* Bell polynomial module.  The module has a class for complete and partial Bell polynomials.

Begin by creating an instance of the Bell polynomial class.

In [12]:
my_bell = wp.bell()

Now generate some Bell polynomials.  By definition, $B_0 = 1$, so insert that.

In [13]:
n = 5
b = np.random.default_rng().uniform(low=1, high=10, size=n)
b = np.insert(b, 0, 1.)

Now invert the Bell polynomials to find the *x's* that would give rise to those Bell polynomials.

In [14]:
x = my_bell.invert(b)

Compute the Bell polynomials from those *x's* and use them to compute the Bell polynomials.  Compare the computed polynomials to the original ones.

In [15]:
bc = my_bell.compute(x)

for n in range(len(x)):
    print(
        "n = {:d}, x[n] = {:.6e}, b[n] = {:.6e}, bc[n] = {:.6e}".format(
            n, x[n], b[n], bc[n]
        )
    )

n = 0, x[n] = 0.000000e+00, b[n] = 1.000000e+00, bc[n] = 1.000000e+00
n = 1, x[n] = 9.857381e+00, b[n] = 9.857381e+00, bc[n] = 9.857381e+00
n = 2, x[n] = -9.336011e+01, b[n] = 3.807854e+00, bc[n] = 3.807854e+00
n = 3, x[n] = 1.806513e+03, b[n] = 3.475969e+00, bc[n] = 3.475969e+00
n = 4, x[n] = -5.238279e+04, b[n] = 7.426402e+00, bc[n] = 7.426402e+00
n = 5, x[n] = 2.025388e+06, b[n] = 8.260705e+00, bc[n] = 8.260705e+00


Now we study the partial Bell polynomial class.  First create an instance.

In [16]:
my_partial_bell = wp.partial_bell()

Now compute the partial Bell polynomials from the *x's* above.  The return is a two-dimensional array with the indices *n* and *k* giving $B_{n,k}(x_1, ..., x_{n-k+1})$.

In [17]:
pbc = my_partial_bell.compute(x)

Check the results by noting that the complete Bell polynomials $B_n$ are given by $B_n(x_1, ..., x_n) = \sum_{k=1}^n B_{n,k}(x_1, ..., x_{n-k+1})$.  We note that $B_{0,0} = 1$ but $B_{n,0} = 0$ for $n > 0$.

In [18]:
for n in range(len(x)):
    bn = 0
    for k in range(0, n+1):
        bn += pbc[n, k]
    print(
        "n = {:d}, x[n] = {:.6e}, bc[n] = {:.6e}, bn = {:.6e}".format(
            n, x[n], bc[n], bn
        )
    )

n = 0, x[n] = 0.000000e+00, bc[n] = 1.000000e+00, bn = 1.000000e+00
n = 1, x[n] = 9.857381e+00, bc[n] = 9.857381e+00, bn = 9.857381e+00
n = 2, x[n] = -9.336011e+01, bc[n] = 3.807854e+00, bn = 3.807854e+00
n = 3, x[n] = 1.806513e+03, bc[n] = 3.475969e+00, bn = 3.475969e+00
n = 4, x[n] = -5.238279e+04, bc[n] = 7.426402e+00, bn = 7.426402e+00
n = 5, x[n] = 2.025388e+06, bc[n] = 8.260705e+00, bn = 8.260705e+00


As a final exercise, we note that $x_n = B_{n,1}$.  Here we check that.

In [19]:
for n in range(len(x)):
    print("n = {:d}, x[n] = {:.6e}, pbc[n,1] = {:.6e}".format(n, x[n], pbc[n, 1]))

n = 0, x[n] = 0.000000e+00, pbc[n,1] = 0.000000e+00
n = 1, x[n] = 9.857381e+00, pbc[n,1] = 9.857381e+00
n = 2, x[n] = -9.336011e+01, pbc[n,1] = -9.336011e+01
n = 3, x[n] = 1.806513e+03, pbc[n,1] = 1.806513e+03
n = 4, x[n] = -5.238279e+04, pbc[n,1] = -5.238279e+04
n = 5, x[n] = 2.025388e+06, pbc[n,1] = 2.025388e+06
